In [1]:
import boto3
import pandas as pd
import numpy as np
import io
import sys
import os
import seaborn as sns
import sqlparse

# local libraries
sys.path.append(os.path.abspath(os.path.join(sys.path[0], '..', 'src', 'lib')))
#sys.path.append('/mnt/src/lib') # Domino
import market_maker_training

# options
pd.options.display.max_rows = 100
pd.options.display.max_columns = 1000
pd.options.mode.chained_assignment = None
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
coin_pair_dict = {'target':'btcusdt',
                  'alt':'ethusdt',
                  'through':'ethbtc'}
feature_minutes_list = [1, 3, 5, 8, 11, 14, 18, 22, 30, 40, 50, 60, 120, 240, 480, 960, 2000]
feature_minutes_list = [5]
trade_window_list=[6]

### Print SQL

In [4]:
mm_training = market_maker_training.BinanceTraining(coin_pair_dict, feature_minutes_list, trade_window_list)
bsql = mm_training.training_data_sql
bsql = bsql.replace('\n','')
print(sqlparse.format(bsql, reindent=True, keyword_case='upper'))

WITH target_btcusdt AS
  (SELECT coin_pair AS btcusdt_coin_partition,
          to_timestamp(trade_minute * 60) AS btcusdt_trade_datetime,
          DATE(to_timestamp(trade_minute * 60)) AS btcusdt_trade_date,
          trade_minute AS btcusdt_trade_minute,
          bids_top_price AS btcusdt_bids_top_price,
          (asks_top_price - bids_top_price) AS btcusdt_bid_ask_spread,
          (asks_top_price + bids_top_price) / 2 AS btcusdt_bid_ask_average_price,
          bids_cum_5000_weighted_avg AS btcusdt_bids_cum_5000_weighted_avg,
          bids_cum_10000_weighted_avg AS btcusdt_bids_cum_10000_weighted_avg,
          bids_cum_20000_weighted_avg AS btcusdt_bids_cum_20000_weighted_avg,
          bids_cum_50000_weighted_avg AS btcusdt_bids_cum_50000_weighted_avg,
          bids_cum_100000_weighted_avg AS btcusdt_bids_cum_100000_weighted_avg,
          bids_cum_200000_weighted_avg AS btcusdt_bids_cum_200000_weighted_avg,
          bids_cum_5000_weighted_std AS btcusdt_bids_cum_5000_weigh